In [2]:
import pandas as pd
import numpy as np

In [3]:
ul = pd.read_csv('use_log.csv')
cust_m = pd.read_csv('customer_master.csv')
class_m = pd.read_csv('class_master.csv')
camp_m = pd.read_csv('campaign_master.csv')

- user_log : 이용 이력 데이터
- cust_m : 회원 정보 데이터 
- 탈퇴한 회원을 포함한 모든 회원에 대한 정보 / end date가 비어져있으면 탈퇴하지 않은 회원
- class_m : 회원구분 데이터(종일 / 주간 / 야간 )
- camp_m : 프로모션구분 데이터(무료 / 반액할인 / 일반권)

In [4]:
display(ul.head(), cust_m.head(), class_m.head(), camp_m.head())
#동시에 출력

log_id customer_id     usedate
0  L00000049012330    AS009373  2018-04-01
1  L00000049012331    AS015315  2018-04-01
2  L00000049012332    AS040841  2018-04-01
3  L00000049012333    AS046594  2018-04-01
4  L00000049012334    AS073285  2018-04-01

customer_id   name class gender           start_date end_date campaign_id  \
0    OA832399   XXXX   C01      F  2015-05-01 00:00:00      NaN         CA1   
1    PL270116  XXXXX   C01      M  2015-05-01 00:00:00      NaN         CA1   
2    OA974876  XXXXX   C01      M  2015-05-01 00:00:00      NaN         CA1   
3    HD024127  XXXXX   C01      F  2015-05-01 00:00:00      NaN         CA1   
4    HD661448  XXXXX   C03      F  2015-05-01 00:00:00      NaN         CA1   

   is_deleted  
0           0  
1           0  
2           0  
3           0  
4           0

class class_name  price
0   C01       0_종일  10500
1   C02       1_주간   7500
2   C03       2_야간   6000

campaign_id campaign_name
0         CA1          2_일반
1         CA2     0_입회비반액할인
2         CA3       1_입회비무료

고객현황 데이터인 cust_m 에 class_m 데이터를 결합하여 새로운 데이터 프레임을 생성

In [5]:
cust_join = pd.merge(cust_m, class_m, on='class', how='left')
cust_join.head()

customer_id   name class gender           start_date end_date campaign_id  \
0    OA832399   XXXX   C01      F  2015-05-01 00:00:00      NaN         CA1   
1    PL270116  XXXXX   C01      M  2015-05-01 00:00:00      NaN         CA1   
2    OA974876  XXXXX   C01      M  2015-05-01 00:00:00      NaN         CA1   
3    HD024127  XXXXX   C01      F  2015-05-01 00:00:00      NaN         CA1   
4    HD661448  XXXXX   C03      F  2015-05-01 00:00:00      NaN         CA1   

   is_deleted class_name  price  
0           0       0_종일  10500  
1           0       0_종일  10500  
2           0       0_종일  10500  
3           0       0_종일  10500  
4           0       2_야간   6000

In [6]:
cust_join = pd.merge(cust_join, camp_m, on='campaign_id', how='left')
cust_join.head()

customer_id   name class gender           start_date end_date campaign_id  \
0    OA832399   XXXX   C01      F  2015-05-01 00:00:00      NaN         CA1   
1    PL270116  XXXXX   C01      M  2015-05-01 00:00:00      NaN         CA1   
2    OA974876  XXXXX   C01      M  2015-05-01 00:00:00      NaN         CA1   
3    HD024127  XXXXX   C01      F  2015-05-01 00:00:00      NaN         CA1   
4    HD661448  XXXXX   C03      F  2015-05-01 00:00:00      NaN         CA1   

   is_deleted class_name  price campaign_name  
0           0       0_종일  10500          2_일반  
1           0       0_종일  10500          2_일반  
2           0       0_종일  10500          2_일반  
3           0       0_종일  10500          2_일반  
4           0       2_야간   6000          2_일반

In [7]:
#공통컬럼 기준으로 잘 합쳐졌는지 확인
print(len(cust_m))
print(len(cust_join))

4192
4192


In [8]:
# cust_join = pd.merge(cust_join, ul, on='customer_id', how='left')
# cust_join.head()

In [9]:
cust_join.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4192 entries, 0 to 4191
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customer_id    4192 non-null   object
 1   name           4192 non-null   object
 2   class          4192 non-null   object
 3   gender         4192 non-null   object
 4   start_date     4192 non-null   object
 5   end_date       1350 non-null   object
 6   campaign_id    4192 non-null   object
 7   is_deleted     4192 non-null   int64 
 8   class_name     4192 non-null   object
 9   price          4192 non-null   int64 
 10  campaign_name  4192 non-null   object
dtypes: int64(2), object(9)
memory usage: 393.0+ KB


In [10]:
cust_join.isna().sum()

customer_id         0
name                0
class               0
gender              0
start_date          0
end_date         2842
campaign_id         0
is_deleted          0
class_name          0
price               0
campaign_name       0
dtype: int64

원본인 cust_m과 결합된 새로운데이터프레임인 cust_join의 행의 수가 같으므로 열 기준으로 결합이 잘됨

- 결측치를 확인하는 이유는 결합할 때 키가 없거나 결합이 잘못되면 자동으로 결측치가 들어가기 때문 -> 그러므로 결합 후에는 결측치를 확인해주는게 좋음 
- 원래 결측치가 있던 end_data에만 결측치가 있으므로 데이터가 잘 들어가져있다라고 확인됨

In [11]:
cust_join['customer_id'].nunique() #우리 고객의 수가 4192명이 맞구나

4192

#### cust_join 데이터프레임 분석

내가 이 데이터프레임을 봤을 때 어떤 궁금한 점을 뽑아낼수있을까?

<예시>
- 어떤 종류의 회원이 있고 어떤 캠페인으로 많이 가입했는지?
- 언제 가입/탈퇴한 회원이 많은지?
- 남녀 비율은 어떤지?
- 탈퇴한 회원들은 얼만큼의 기간동안 스포트센처를 다녔는지?

In [12]:
cust_join.head()

customer_id   name class gender           start_date end_date campaign_id  \
0    OA832399   XXXX   C01      F  2015-05-01 00:00:00      NaN         CA1   
1    PL270116  XXXXX   C01      M  2015-05-01 00:00:00      NaN         CA1   
2    OA974876  XXXXX   C01      M  2015-05-01 00:00:00      NaN         CA1   
3    HD024127  XXXXX   C01      F  2015-05-01 00:00:00      NaN         CA1   
4    HD661448  XXXXX   C03      F  2015-05-01 00:00:00      NaN         CA1   

   is_deleted class_name  price campaign_name  
0           0       0_종일  10500          2_일반  
1           0       0_종일  10500          2_일반  
2           0       0_종일  10500          2_일반  
3           0       0_종일  10500          2_일반  
4           0       2_야간   6000          2_일반

In [13]:
cust_join.class_name.value_counts()

0_종일    2045
2_야간    1128
1_주간    1019
Name: class_name, dtype: int64

In [14]:
cust_join.groupby('class_name')['customer_id'].count()

class_name
0_종일    2045
1_주간    1019
2_야간    1128
Name: customer_id, dtype: int64

In [15]:
#회원권 종류별로 비율확인
round((cust_join.groupby('class_name')['customer_id'].count() / cust_join.shape[0]) * 100, 1)

class_name
0_종일    48.8
1_주간    24.3
2_야간    26.9
Name: customer_id, dtype: float64

In [16]:
#프로모션 현황 비율확인
round((cust_join.groupby('campaign_name')['customer_id'].count() / cust_join.shape[0]) * 100, 1)

campaign_name
0_입회비반액할인    15.5
1_입회비무료      11.7
2_일반         72.8
Name: customer_id, dtype: float64

In [17]:
#성별 현황
cust_join.groupby('gender')['customer_id'].count()

gender
F    1983
M    2209
Name: customer_id, dtype: int64

In [18]:
#성별 비율
round((cust_join.groupby('gender')['customer_id'].count() / cust_join.shape[0]) * 100, 1)

gender
F    47.3
M    52.7
Name: customer_id, dtype: float64

In [19]:
#탈퇴자 현황
cust_join.groupby('is_deleted')['customer_id'].count()

is_deleted
0    2842
1    1350
Name: customer_id, dtype: int64

In [20]:
#탈퇴자 비율
round((cust_join.groupby('is_deleted')['customer_id'].count() / cust_join.shape[0]) * 100, 1)

is_deleted
0    67.8
1    32.2
Name: customer_id, dtype: float64

데이터를 분석하면서 항상 궁금한 점을 생각해보기
- 남자가 더 많이 가입한 이유가 있을까?
- 탈퇴한사람들은 왜 탈퇴했을까? 

### 데이터 조건별로 뽑아보기

최신데이터를 뽑아보기
- 2019년 3월 이용자 뽑기 -> end date가 2019.3.31 이후거나 탈퇴한 적이 없어야 함

In [21]:
cust_join['end_date'] = pd.to_datetime(cust_join['end_date'])
cust_new = cust_join[(cust_join['end_date']>=pd.to_datetime('20190331'))| (cust_join['end_date'].isnull())]
cust_new.head()

customer_id   name class gender           start_date end_date campaign_id  \
0    OA832399   XXXX   C01      F  2015-05-01 00:00:00      NaT         CA1   
1    PL270116  XXXXX   C01      M  2015-05-01 00:00:00      NaT         CA1   
2    OA974876  XXXXX   C01      M  2015-05-01 00:00:00      NaT         CA1   
3    HD024127  XXXXX   C01      F  2015-05-01 00:00:00      NaT         CA1   
4    HD661448  XXXXX   C03      F  2015-05-01 00:00:00      NaT         CA1   

   is_deleted class_name  price campaign_name  
0           0       0_종일  10500          2_일반  
1           0       0_종일  10500          2_일반  
2           0       0_종일  10500          2_일반  
3           0       0_종일  10500          2_일반  
4           0       2_야간   6000          2_일반

In [22]:
cust_new['end_date'].unique()

array([                          'NaT', '2019-03-31T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [23]:
#어떤 회원권을 많이 사용하고 있는지 확인
cust_new.groupby('class_name')['customer_id'].count()

class_name
0_종일    1444
1_주간     696
2_야간     813
Name: customer_id, dtype: int64

In [24]:
#회원권 비율 확인
round((cust_new.groupby('class_name')['customer_id'].count() / cust_join.shape[0]) * 100, 1)

class_name
0_종일    34.4
1_주간    16.6
2_야간    19.4
Name: customer_id, dtype: float64

In [25]:
#프로모션 비율 확인
round((cust_new.groupby('campaign_name')['customer_id'].count() / cust_join.shape[0]) * 100, 1)

campaign_name
0_입회비반액할인     7.4
1_입회비무료       5.8
2_일반         57.3
Name: customer_id, dtype: float64

In [26]:
#성별 비율 확인
round((cust_new.groupby('gender')['customer_id'].count() / cust_join.shape[0]) * 100, 1)

gender
F    33.4
M    37.0
Name: customer_id, dtype: float64

<데이터 분석 해석>
- 회원권과 성별 변수에 대해서는 전체 데이터에 대해 뽑았던 것과 비율이 크게 다르지 않음 특정 회원권이나 성별이 탈퇴한 것은 아님
- 전체 데이터에서 프로모션 비율 중일반가입이 70% 정도였는데, 최신 사용자 데이터에서는 일반가입이 80%(약 10% 차이), 프로모션으로 가입한 사람은 일반 가입한 사람보다 계속 이용할 가능성이 낮을수 있지 않을까 생각해볼수있다.

#### 이용이력 데이터 분석

In [27]:
ul.head()

log_id customer_id     usedate
0  L00000049012330    AS009373  2018-04-01
1  L00000049012331    AS015315  2018-04-01
2  L00000049012332    AS040841  2018-04-01
3  L00000049012333    AS046594  2018-04-01
4  L00000049012334    AS073285  2018-04-01

회원이 스포츠센터를 이용할 때 시간이 찍히기 때문에 시간적 요소를 분석해볼수 있겠다

<예시>
- 한달동안 이용 횟수의 변화
- 회원이 스포트센터를 정기적으로 이용하는지 아무때나 오는지
- 회원들이 월 별 스포츠센터를 이용하는 평균 수치, 중앙값, 최대,최소값

In [28]:
ul['usedate'] = pd.to_datetime(ul['usedate']) #데이터를 날짜형 데이터로 변환

In [29]:
ul['년월'] = ul['usedate'].dt.strftime('%Y%m')

In [30]:
ul.head()

log_id customer_id    usedate      년월
0  L00000049012330    AS009373 2018-04-01  201804
1  L00000049012331    AS015315 2018-04-01  201804
2  L00000049012332    AS040841 2018-04-01  201804
3  L00000049012333    AS046594 2018-04-01  201804
4  L00000049012334    AS073285 2018-04-01  201804

In [31]:
#월별, 고객 id별 이용횟수 카운트
ul_month = ul.groupby(['년월','customer_id'], as_index=False).count()
# as_index=False 없으면 년월이 index로 잡힘

#### 월별 고객의 스포츠 센터

In [32]:
#컬럼 이름 변경
ul_month.rename(columns={'log_id' : 'cnt'}, inplace='True')
#inplace='True' 변경 데이터 적용
#usedate 열 삭제
ul_month.drop('usedate', axis=1, inplace=True)
ul_month.head()

년월 customer_id  cnt
0  201804    AS002855    4
1  201804    AS009013    2
2  201804    AS009373    3
3  201804    AS015315    6
4  201804    AS015739    7

In [35]:
#고객별 월평균 사용횟수
ul_cust = ul_month.groupby('customer_id')['cnt'].agg([np.mean, np.median, np.max, np.min])
ul_cust = ul_cust.reset_index(drop=False)
ul_cust.head()

customer_id      mean  median  amax  amin
0    AS002855  4.500000     5.0     7     2
1    AS008805  4.000000     4.0     8     1
2    AS009013  2.000000     2.0     2     2
3    AS009373  5.083333     5.0     7     3
4    AS015233  7.545455     7.0    11     4

#### 정기적으로 스포트센터를 사용하는 고객인지 확인

- 매주 같은 요일에 스포츠센터에 방문했는지 안했는지로 판단해보기
- 월/요일별로 방문한 횟수의 최대값이 4이상인지 요일이 하나라도 있으면 이 회원은 정기 사용자로 판단


In [36]:
#요일 뽑기
#weekday : 월요일 - 0, 일요일 - 6
ul['weekday'] = ul['usedate'].dt.weekday
ul.head()

log_id customer_id    usedate      년월  weekday
0  L00000049012330    AS009373 2018-04-01  201804        6
1  L00000049012331    AS015315 2018-04-01  201804        6
2  L00000049012332    AS040841 2018-04-01  201804        6
3  L00000049012333    AS046594 2018-04-01  201804        6
4  L00000049012334    AS073285 2018-04-01  201804        6

In [37]:
ul_week = ul.groupby(['customer_id', '년월', 'weekday'], as_index=False).count()

In [39]:
ul_week.head()

customer_id      년월  weekday  log_id  usedate
0    AS002855  201804        5       4        4
1    AS002855  201805        2       1        1
2    AS002855  201805        5       4        4
3    AS002855  201806        5       5        5
4    AS002855  201807        1       1        1

In [40]:
ul_week.rename(columns={'log_id':'cnt'}, inplace=True)

In [42]:
ul_week.head()

customer_id      년월  weekday  cnt  usedate
0    AS002855  201804        5    4        4
1    AS002855  201805        2    1        1
2    AS002855  201805        5    4        4
3    AS002855  201806        5    5        5
4    AS002855  201807        1    1        1

In [43]:
del ul_week['usedate']
ul_week.head()

customer_id      년월  weekday  cnt
0    AS002855  201804        5    4
1    AS002855  201805        2    1
2    AS002855  201805        5    4
3    AS002855  201806        5    5
4    AS002855  201807        1    1